In [25]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
import pandas as pd
import time
import telegram_send

In [26]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

#Luneal
luneael ='https://etherscan.io/txs?a=0x8b3f4eb783270aefaac9238ac1d165a433c8fbf3&f=2'
req_luneael = Request(luneael, headers= headers)

#Luneael 2
luneael2 ='https://etherscan.io/txs?a=0xb8659f9034fdb8608598f7295b6b82869c379510=2'
req_luneael2 = Request(luneael2, headers= headers)

#Wickd
wickd ='https://etherscan.io/txs?a=0x820880c9040041d6a514e069ec0a2f37997325d4&f=2'
req_wickd = Request(wickd, headers=headers)

#Sneakyninjapants
sneakyninjapants ='https://etherscan.io/txs?a=0xbdd95abe8a7694ccd77143376b0fbea183e6a740&f=2'
req_sneakyninjapants = Request(sneakyninjapants, headers=headers)

In [27]:
def scraper_etherscan ():
    #Luneal
    response_luneael = urlopen(req_luneael, timeout=20).read()
    response_luneael_close = urlopen(req_luneael, timeout=20).close()
    page_soup_luneael = soup(response_luneael, "html.parser")
    Transfers_info_table_luneael = page_soup_luneael.find("table", {"class": "table table-hover"})
    luneael_data = pd.read_html(str(Transfers_info_table_luneael))[0]
    df_luneael = pd.DataFrame(luneael_data)

    #Luneal2
    response_luneael2 = urlopen(req_luneael2, timeout=20).read()
    response_luneael2_close = urlopen(req_luneael2, timeout=20).close()
    page_soup_luneael2 = soup(response_luneael2, "html.parser")
    Transfers_info_table_luneael2 = page_soup_luneael2.find("table", {"class": "table table-hover"})
    luneael2_data = pd.read_html(str(Transfers_info_table_luneael2))[0]
    df_luneael2 = pd.DataFrame(luneael2_data)

    #Wickd
    response_wickd = urlopen(req_wickd, timeout=20).read()
    response_wickd_close = urlopen(req_wickd, timeout=20).close()
    page_soup_wickd = soup(response_wickd, "html.parser")
    Transfers_info_table_wickd = page_soup_wickd.find("table", {"class": "table table-hover"})
    wickd_data = pd.read_html(str(Transfers_info_table_wickd))[0]
    df_wickd = pd.DataFrame(wickd_data)

    #sneakyninjapants
    response_sneakyninjapants = urlopen(req_sneakyninjapants, timeout=20).read()
    response_sneakyninjapants_close = urlopen(req_sneakyninjapants, timeout=20).close()
    page_soup_sneakyninjapants = soup(response_sneakyninjapants, "html.parser")
    Transfers_info_table_sneakyninjapants = page_soup_sneakyninjapants.find("table", {"class": "table table-hover"})
    sneakyninjapants_data = pd.read_html(str(Transfers_info_table_sneakyninjapants))[0]
    df_sneakyninjapants = pd.DataFrame(sneakyninjapants_data)

    conso = [df_luneael, df_luneael2, df_wickd, df_sneakyninjapants]

    dfs = pd.concat(conso)

    dfs.drop(['Unnamed: 0', 'Block','Unnamed: 10'], axis=1, inplace=True)
    dfs.rename(columns={"Method": "Nature", "Unnamed: 4": "Time", "Unnamed: 6" : "In/Out","Unnamed: 9": "Gas fees"}, inplace=True )

    dfs["Nature"] = dfs["Nature"].str.lower()
    dfs['Value'] = dfs['Value'].str.replace(r'\ Ether', '')
    dfs['Time'] = dfs['Time'].str.replace(r'\ ago', '')

    dfs['Hours'] = dfs['Time'].str[-15:-10]
    dfs['Mins'] = dfs['Time'].str[-7:-4]
    dfs['Hours'] = dfs['Hours'].str.extract('(\d+)', expand=False)
    dfs['Mins'] = dfs['Mins'].str.extract('(\d+)', expand=False)
    dfs['Hours'] = dfs['Hours'].fillna(0)
    dfs["Hours"] = pd.to_numeric(dfs["Hours"], downcast="float")
    dfs["Mins"] = pd.to_numeric(dfs["Mins"], downcast="float")


    conditions = [
        (dfs['Hours'] == 0) & (dfs['Mins']<=2),
        (dfs['Nature'].str.contains("mint").any())]

    #time  = dfs.iloc[0,2]
    #owner = dfs.iloc[0,[3]]
    #destination = dfs.iloc[0,[5]]
    #value = dfs.iloc[0,[6]]

    last_transaction = dfs.iloc[0,[3,2,5,6]]

    message = last_transaction.to_string()

    #dfs.reset_index(inplace=True)
    #dfs = dfs.to_json()

    if conditions is True:
        telegram_send.send(messages=[message])

    else:
        telegram_send.send(messages=[message])

while True:
    scraper_etherscan()
    time.sleep(10)

/var/folders/d6/_6wp3vwx1cg1prk_b2khmx240000gn/T/ipykernel_61016/3407575292.py:42: FutureWarning: The default value of regex will change from True to False in a future version.
  dfs['Value'] = dfs['Value'].str.replace(r'\ Ether', '')
/var/folders/d6/_6wp3vwx1cg1prk_b2khmx240000gn/T/ipykernel_61016/3407575292.py:43: FutureWarning: The default value of regex will change from True to False in a future version.
  dfs['Time'] = dfs['Time'].str.replace(r'\ ago', '')


KeyboardInterrupt: 

In [ ]:
#message = print(f'"Who :"{owner}\n "To :" {destination}\n "Time :"{time}"ago" \n "Value :"{value}')